In [1]:
from pyspark import SparkConf,SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyAppnew").setMaster("local"))
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

def path_step(item):
    prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
    return (next_v, prev_d + tuple([next_v]))

n = 4  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge)
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

x = 12
z = 34

paths = sc.parallelize([(x, tuple([x])) ]).partitionBy(n)

d = 0

while True:
    d += 1    
    paths = paths.join(forward_edges, n).map(path_step)
    if paths.filter(lambda x: x[0] == z).count() > 0:
        break     


final_paths = paths.filter(lambda value: value[0] == z).map(lambda value: value[1])              

min_path = final_paths.take(1)[0]

print(','.join(map(str, min_path)))

12,422,53,52,107,20,23,274,34
